In [1]:
!pip install transformers datasets torch scikit-learn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [35]:
import pandas as pd
df = pd.read_csv("/labeled.csv")
print(df.head())
df['toxic'] = df['toxic'].astype(int)


                                             comment  toxic
0               Верблюдов-то за что? Дебилы, бл...\n    1.0
1  Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                          Собаке - собачья смерть\n    1.0
3  Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4  тебя не убедил 6-страничный пдф в том, что Скр...    1.0


In [37]:
# удаление спецсимволов, удаление стоп-слов (например, "и", "в", "на")

df['comment'] = df['comment'].fillna('').str.replace(r'[^\w\s]', '', regex=True)

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
russian_stopwords = set(stopwords.words("russian"))

df['comment'] = df['comment'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in russian_stopwords])
)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
# разделение данных

from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['comment'], df['toxic'], test_size=0.2, random_state=42)


In [41]:
# подгрузка моделей

from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruRoberta-large")
model = AutoModelForSequenceClassification.from_pretrained("sberbank-ai/ruRoberta-large", num_labels=2)


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [42]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)


In [43]:
import torch

class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = ToxicDataset(train_encodings, train_labels.tolist())
val_dataset = ToxicDataset(val_encodings, val_labels.tolist())


In [45]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=5,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_steps=50,
    save_strategy="epoch",
    save_total_limit=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./results",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-45-8496827461c9>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [46]:
print(train_dataset[0])
print(train_labels[:5])


{'input_ids': tensor([    1,  1064, 10028, 21161,  7031, 12201, 35512, 17883,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [47]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.247500,0.222763
2,0.200500,0.219729
3,0.149100,0.226905
4,0.055500,0.340524
5,0.051900,0.408594


TrainOutput(global_step=3605, training_loss=0.1647130130959388, metrics={'train_runtime': 4998.4292, 'train_samples_per_second': 11.533, 'train_steps_per_second': 0.721, 'total_flos': 1.343029585976064e+16, 'train_loss': 0.1647130130959388, 'epoch': 5.0})

In [48]:
predictions = trainer.predict(val_dataset)

import numpy as np
from sklearn.metrics import classification_report

preds = np.argmax(predictions.predictions, axis=1)
true_labels = val_labels.tolist()


In [49]:
report = classification_report(true_labels, preds, target_names=["Non-Toxic", "Toxic"])
print(report)


              precision    recall  f1-score   support

   Non-Toxic       0.94      0.95      0.95      1944
       Toxic       0.90      0.87      0.88       939

    accuracy                           0.93      2883
   macro avg       0.92      0.91      0.91      2883
weighted avg       0.93      0.93      0.93      2883



In [50]:
model.save_pretrained("./toxicity_model")
tokenizer.save_pretrained("./toxicity_model")


('./toxicity_model/tokenizer_config.json',
 './toxicity_model/special_tokens_map.json',
 './toxicity_model/vocab.json',
 './toxicity_model/merges.txt',
 './toxicity_model/added_tokens.json',
 './toxicity_model/tokenizer.json')

In [55]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Загрузка сохраненной модели
tokenizer = AutoTokenizer.from_pretrained("./toxicity_model")
model = AutoModelForSequenceClassification.from_pretrained("./toxicity_model")

example_text = "Я тебя убью собака."

inputs = tokenizer(
    example_text,
    return_tensors="pt",  # Подготовка для PyTorch
    truncation=True,
    padding=True,
    max_length=128
)


model.eval()  # Перевод модели в режим оценки
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)


predicted_class = torch.argmax(probabilities, dim=1).item()
print(f"Класс: {'Токсичный' if predicted_class == 1 else 'Нетоксичный'}")
print(f"Вероятности: {probabilities.numpy()}")


Класс: Токсичный
Вероятности: [[0.00243815 0.9975618 ]]


In [52]:
test_texts = [
    "Твои знания про ИИ — на уровне первого класса, не позорься.",
    "Кому вообще интересен этот бред про искусственный интеллект?",
    "Пустая болтовня, лучше бы молчали.",
    "Автор явно сам не понимает, что говорит, фу.",
    "Хватит продвигать этот ненужный ИИ, он никому не нужен.",
    "Что за дичь ты несёшь? Никакого смысла!",
    "ИИ — это мусор, и все, кто его изучают, тоже.",
    "Эти ролики — пустая трата времени, удаляй канал!",
    "Автор канала — полный дилетант, даже слушать смешно.",
    "Твои объяснения хуже, чем школьный урок, отстой!",

    "Хорошее видео, спасибо за старание!",
    "Интересно, как это работает, расскажите подробнее!",
    "ИИ выглядит круто, но хотелось бы примеров попроще.",
    "Лайк за старания, но рассказывай немного медленнее.",
    "Обожаю твой контент, продолжай в том же духе!",
    "Неплохо, но можно объяснять чуть доступнее для новичков.",
    "Здорово, что технологии так быстро развиваются, спасибо за информацию.",
    "Тема интересная, буду разбираться, спасибо!",
    "Крутая подача, но хотелось бы примеров побольше.",
    "ИИ реально помогает в жизни, удивительно!",

]

for text in test_texts:
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predicted_class = torch.argmax(probabilities, dim=1).item()
        print(f"Текст: {text}")
        print(f"Класс: {'Токсичный' if predicted_class == 1 else 'Нетоксичный'}")
        print(f"Вероятности: {probabilities.numpy()}\n")


Текст: Твои знания про ИИ — на уровне первого класса, не позорься.
Класс: Токсичный
Вероятности: [[0.29120713 0.7087929 ]]

Текст: Кому вообще интересен этот бред про искусственный интеллект?
Класс: Токсичный
Вероятности: [[0.06129692 0.93870306]]

Текст: Пустая болтовня, лучше бы молчали.
Класс: Токсичный
Вероятности: [[0.00310004 0.99689996]]

Текст: Автор явно сам не понимает, что говорит, фу.
Класс: Нетоксичный
Вероятности: [[0.6395855  0.36041442]]

Текст: Хватит продвигать этот ненужный ИИ, он никому не нужен.
Класс: Токсичный
Вероятности: [[0.00369819 0.9963018 ]]

Текст: Что за дичь ты несёшь? Никакого смысла!
Класс: Токсичный
Вероятности: [[0.01604938 0.9839506 ]]

Текст: ИИ — это мусор, и все, кто его изучают, тоже.
Класс: Токсичный
Вероятности: [[0.00709733 0.9929027 ]]

Текст: Эти ролики — пустая трата времени, удаляй канал!
Класс: Токсичный
Вероятности: [[0.00583122 0.99416876]]

Текст: Автор канала — полный дилетант, даже слушать смешно.
Класс: Нетоксичный
Вероятности: [[